<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/6%20-%20Generatori%20e%20Images%20Augmentation/images_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Images Augmentation

In [5]:
!wget https://profession.ai/datasets/dessert_small.zip
!unzip -qq dessert_small.zip

--2020-06-29 11:12:10--  https://profession.ai/datasets/dessert_small.zip
Resolving profession.ai (profession.ai)... 13.224.166.2, 13.224.166.63, 13.224.166.122, ...
Connecting to profession.ai (profession.ai)|13.224.166.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52003499 (50M) [application/zip]
Saving to: ‘dessert_small.zip’

dessert_small.zip   100%[===================>]  49.59M  15.3MB/s    in 4.1s    

2020-06-29 11:12:15 (12.1 MB/s) - ‘dessert_small.zip’ saved [52003499/52003499]



In [6]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
DATASET_DIR = "dessert_small/"
BATCH_SIZE = 64

TOT_IMAGES = 1000 
IMG_SIZE = (228, 228)

In [8]:
datagen = ImageDataGenerator(validation_split=0.2, 
                            rescale=1./255,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True)

In [9]:
train_generator = datagen.flow_from_directory(
        DATASET_DIR+"train", 
        target_size=IMG_SIZE, 
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training')

valid_generator = datagen.flow_from_directory(
        DATASET_DIR+"train",
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation')


test_generator = datagen.flow_from_directory(
        DATASET_DIR+"test",
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

print(train_generator.class_indices)

Found 800 images belonging to 5 classes.
Found 200 images belonging to 5 classes.
Found 1250 images belonging to 5 classes.
{'cannoli': 0, 'gelato': 1, 'panna_cotta': 2, 'tiramisu': 3, 'torta_di_mele': 4}


In [24]:
#regularizer = tf.keras.regularizers.L1L2(l1=0.01, l2=0.1)
regularizer = tf.keras.regularizers.L1L2(l2=0.1)

model = Sequential()

model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)))
model.add(Conv2D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Dropout(0.4))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=3, strides=3))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(5, activation='softmax'))

print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 228, 228, 64)      1792      
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 228, 228, 64)      36928     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 76, 76, 64)        0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 76, 76, 64)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 76, 76, 32)        18464     
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 76, 76, 32)        9248      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 25, 25, 32)       

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                              min_delta=0.01,
                              patience=3,
                              restore_best_weights=True)

model.fit(
        train_generator,
        steps_per_epoch=int(TOT_IMAGES*0.8 // BATCH_SIZE),
        validation_data=valid_generator, 
        validation_steps=int(TOT_IMAGES*0.2 // BATCH_SIZE),
        epochs=100)

Epoch 1/100
12/12 [==============================] - 16s 1s/step - loss: 1.6478 - accuracy: 0.2228 - val_loss: 1.6089 - val_accuracy: 0.2344
Epoch 2/100
12/12 [==============================] - 16s 1s/step - loss: 1.6087 - accuracy: 0.2160 - val_loss: 1.6093 - val_accuracy: 0.1979
Epoch 3/100
12/12 [==============================] - 16s 1s/step - loss: 1.6080 - accuracy: 0.1929 - val_loss: 1.6073 - val_accuracy: 0.1979
Epoch 4/100
12/12 [==============================] - 15s 1s/step - loss: 1.6000 - accuracy: 0.2120 - val_loss: 1.6028 - val_accuracy: 0.2865
Epoch 5/100
12/12 [==============================] - 15s 1s/step - loss: 1.5949 - accuracy: 0.2133 - val_loss: 1.6047 - val_accuracy: 0.2292
Epoch 6/100
12/12 [==============================] - 16s 1s/step - loss: 1.5900 - accuracy: 0.2609 - val_loss: 1.5833 - val_accuracy: 0.2500
Epoch 7/100
12/12 [==============================] - 16s 1s/step - loss: 1.5930 - accuracy: 0.2636 - val_loss: 1.5994 - val_accuracy: 0.2396
Epoch 8/100
 